In [ ]:
import numpy as np
import pandas as pd
from helpers import *

# TLDR
Build pubmed queries (that can be turned into RSS feeds) from Google sheet describing parameters.

### Type 1: People search – return articles from people based on name/descriptors
### Type 2: Keyword search – return articles from keywords + journals

Example Google Sheet: https://docs.google.com/spreadsheets/d/108U4eq7zkwbqaygDMlCKe1cp0q0BlJ6Z0beI7Q9Egzs/edit#gid=0

# Motivation

Typical approaches for following the literature aren't that good at differentiating the signal from the noise.

Examples of signal:

- Papers in and around primary field (from any level of journal if it's sufficiently close, but needs to be more relevant as source IF decreases)
- Exciting papers of broad relevance to the field (e.g. cell engineering / cell therapies writ large)
- Papers by key people, such as personal connections and famous scientists

Examples of noise:

- Intractable numbers of papers to screen
- Papers in predatory journals or from obscure people/places
- Non research/review articles (e.g. news, notes, etc.)

## Pubmed search helpful tips

* use `hasabstract` to filter out news, notes, errata, etc.
* use `NOT review[pt]` to filter out reviews; see [HERE](https://pubmed.ncbi.nlm.nih.gov/help/#publication-types)
* use `last X years[dp]` to filter by recency; see [HERE](https://pubmed.ncbi.nlm.nih.gov/help/#filter-strategy-pubdate)

Also –

* Wildcards can be used, like `Nat Rev*[Journal]` for all Nat Rev family journals
* `[1au]` and `[lastau]` flags only work with truncated names like Bhargava HK, not with full name
* `[tiab]` can be used to limit keyword search to title and abstract



# Read Data from Google Sheet
Google sheet here: https://docs.google.com/spreadsheets/d/108U4eq7zkwbqaygDMlCKe1cp0q0BlJ6Z0beI7Q9Egzs/edit#gid=0

In [ ]:
sheet_id = '108U4eq7zkwbqaygDMlCKe1cp0q0BlJ6Z0beI7Q9Egzs'
sheet_name = 'people'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
people_df = pd.read_csv(url, keep_default_na=False)

sheet_id = '108U4eq7zkwbqaygDMlCKe1cp0q0BlJ6Z0beI7Q9Egzs'
sheet_name = 'cell-engineering-keywords'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
keywords = pd.read_csv(url, keep_default_na=False)['Keyword'].to_list()

sheet_id = '108U4eq7zkwbqaygDMlCKe1cp0q0BlJ6Z0beI7Q9Egzs'
sheet_name = 'journal-whitelist'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
journals = pd.read_csv(url, keep_default_na=False)['Journal'].to_list()

# 1. Build the queries

## 1A. People

For each person: `((ANY name+author_position) AND ((ANY affiliation) OR orcid)`

In [ ]:
len(people_df)

79

In [ ]:
people_df['query'] = people_df.apply(lambda row: author_query_from_row(row), axis=1)

people_df = people_df[people_df['Core Field'] == '1']

people_query = ''
i = 0
for _, row in people_df.iterrows():
    people_query += row['query'] 
    if i < len(people_df)-1:
        people_query += ' OR '
    i += 1
    
people_query += ' AND hasabstract'

# Copy to clipboard
print('Query for people (paste into Pubmed):\n')
print(people_query)
!echo "$people_query" | pbcopy

Query for people (paste into Pubmed):

((Wendell A Lim[FAU])) OR ((Hana El-Samad[FAU])) OR ((Kole T Roybal[FAU])) OR ((Alexander Marson[FAU])) OR ((Marcela Maus[FAU])) OR ((Carl H June[FAU] AND June CH[lastau])) OR ((Crystal L Mackall[FAU])) OR ((Michel Sadelain[FAU])) OR ((Justin Eyquem[FAU])) OR ((Wilson W Wong[FAU] AND Wong WW[lastau])) OR ((Steven A Rosenberg[FAU] AND Rosenberg SA[lastau])) OR ((Rogelio A Hernandez-Lopez[FAU])) OR ((Kyle G Daniels[FAU])) OR ((Yvonne Y Chen[FAU] AND Chen YY[lastau])) OR ((Stanley R Riddell[FAU] AND Riddell SR[lastau])) OR ((Darrell J Irvine[FAU] AND Irvine DJ[lastau])) OR ((Gregoire Altan-Bonnet[FAU] AND Altan-Bonnet G[lastau])) OR ((Ira Mellman[FAU] AND Mellman I[lastau])) OR ((Jeffrey A Bluestone[FAU] AND Bluestone JA[lastau])) OR ((Ahmad S Khalil[FAU])) OR ((K Christopher Garcia[FAU] AND Garcia KC[lastau])) OR ((Ronald N Germain[FAU] AND Germain RN[lastau])) OR ((James A Wells[FAU] AND Wells JA[lastau])) OR ((Nicholas P Restifo[FAU] AND Restifo N

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

# Set up the webdriver
options = webdriver.FirefoxOptions()
# options.add_argument("--headless")
driver = webdriver.Firefox(options=options)

# Navigate to the PubMed website
driver.get("https://www.ncbi.nlm.nih.gov/pubmed/")

# Enter the search query in the search box
search_box = driver.find_element("id", "id_term")
search_box.send_keys("Bhargava HK[au]")

# # Submit the search
search_box.submit()

# Wait for the search results to load
# results_table = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.CLASS_NAME, "search-results-list"))
# )
# results_table = WebDriverWait(driver, 10)
# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "full-docsum")))
# # Click the "Create RSS" button

# results_table = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.ID, "search-results-table"))
# )
driver.implicitly_wait(3) # gives an implicit wait for 20 seconds
# rss_button = driver.find_element("id", "search-create-rss")
rss_button = driver.find_element("link text", "Create RSS")
driver.implicitly_wait(1) # gives an implicit wait for 20 seconds

rss_button.click()
rss_button.click()

name_box = driver.find_element("id", "rss-name")
name_box.clear()
name_box.send_keys("Name of Query")

limit_input = Select(driver.find_element("id", "rss-limit"))
limit_input.select_by_visible_text('100')

driver.find_element(By.XPATH,'//button[normalize-space()="Create RSS"]').click()

link_output = driver.find_element("id", "rss-link").get_attribute("value")
print(link_output)
# # Wait for the RSS feed to be generated
# rss_link = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.ID, "rss-link"))
# )

# # Print the RSS link
# print(rss_link.get_attribute("href"))

https://pubmed.ncbi.nlm.nih.gov/rss/search/14OzJS8GjXZPNRSdPCp6dm4klXgl3woxYJievzcBJLreqrvElp/?limit=100&utm_campaign=pubmed-2&fc=20221202155229


In [ ]:
limit_input.type

AttributeError: 'WebElement' object has no attribute 'type'

In [ ]:
rss_button.click()

## 1B. Topics + Journals
`(ANY keyword) AND (ANY journal) AND hasabstract`

In [ ]:
kwquery = "("
for i, kw in enumerate(keywords):
    kwquery += '(\"%s\"[tiab])' % kw
    
    if i < len(keywords) - 1:
        kwquery += ' OR '
        
kwquery += ')'

jquery = '('
for i, jn in enumerate(journals):
    jquery += '(\"%s\"[journal])' % jn
    
    if i < len(journals) - 1:
        jquery += ' OR '
        
jquery += ')'

query = kwquery + ' AND ' + jquery + ' AND hasabstract'

print('Query for keywords + journals (paste into Pubmed):\n')

# Copy to clipboard
# !echo $query | pbcopy
print(query)

Query for keywords + journals (paste into Pubmed):

(("chimeric antigen receptor"[tiab]) OR ("car-t"[tiab]) OR ("car t"[tiab]) OR ("synnotch"[tiab]) OR ("synthetic notch"[tiab]) OR ("cell therapy"[tiab]) OR ("cell engineering"[tiab]) OR ("IL-2"[tiab]) OR ("interleukin-2"[tiab]) OR ("interleukin 2"[tiab]) OR ("cd25"[tiab])) AND (("Cell"[journal]) OR ("Science"[journal]) OR ("Nature*"[journal]) OR ("N Engl J Med"[journal]) OR ("Lancet*"[journal]) OR ("JAMA*"[journal]) OR ("Blood Adv"[journal]) OR ("Blood"[journal]) OR ("Proc Natl Acad Sci U S A"[journal]) OR ("J Clin Oncol"[journal]) OR ("PLoS Computational Biology"[journal]) OR ("PLoS Biology"[journal]) OR ("Science Advances"[journal]) OR ("Neuron"[journal]) OR ("ACS Synthetic Biology"[journal]) OR ("Cell Reports"[journal]) OR ("Cell Systems"[journal]) OR ("Science Signaling"[journal]) OR ("Science Immunology"[journal]) OR ("Front Immunol"[journal]) OR ("Science Translational Medicine"[journal]) OR ("J Clin Invest"[journal]) OR ("J Immu

# Analyze PEOPLE queries

### Check avg N Papers per year for each query (last 5 years)
Note: this is slow because it has to query pubmed for each paper.

In [ ]:
people_df['papers_per_year'] = people_df.apply(lambda row: avg_articles_per_year_last5(row['query']), axis=1)

In [ ]:
people_df.sort_values(by='papers_per_year', ascending=False)

# Analyze Keyword Query

## Check avg N results per year for last 5 years)

In [ ]:
avg_articles_per_year_last5(query)